In [1]:
import os
print(os.getcwd())

C:\Users\Алекс\Downloads\final_steel\final_steel


# Промышленность — доп. задание

**Формулировка**

Вам поручили изучить характеристики процесса обработки стали. Для каждого значения столбца key найдите:

- Длительность времени между первым и последним замером температуры.

- Суммарное время нагрева электродами, то есть сумму значений по всем промежуткам между запусками нагрева электродов.

- Количество запусков нагрева электродами.

- Среднее соотношение потребления активной и реактивной мощности.

По всем полученным столбцам вычислите статистики: средние, минимальные и максимальные значения, медиану и величины 25%- и 75%-квартилей. - **вычисление данных параметров при помощи функции describe()**

**Импорты**

In [2]:
import pandas as pd
import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
import time

Данные состоят из файлов, полученных из разных источников:

- `data_arc.csv` — данные об электродах;

- `data_bulk.csv` — данные о подаче сыпучих материалов (объём);

- `data_bulk_time.csv` *—* данные о подаче сыпучих материалов (время);

- `data_gas.csv` — данные о продувке сплава газом;

- `data_temp.csv` — результаты измерения температуры;

- `data_wire.csv` — данные о проволочных материалах (объём);

- `data_wire_time.csv` — данные о проволочных материалах (время).

Во всех файлах столбец `key` содержит номер партии. В файлах может быть несколько строк с одинаковым значением `key`: они соответствуют разным итерациям обработки.

In [3]:
data_arc = pd.read_csv('data_arc.csv')
data_bulk = pd.read_csv('data_bulk.csv')
data_bulk_time = pd.read_csv('data_bulk_time.csv')
data_gas = pd.read_csv('data_gas.csv')
data_temp = pd.read_csv('data_temp.csv')
data_wire = pd.read_csv('data_wire.csv')
data_wire_time = pd.read_csv('data_wire_time.csv')

***НОВАЯ ТАБЛИЧКА***

In [4]:
df = pd.DataFrame()

## Длительность времени между первым и последним замером температуры. (в секундах)

In [5]:
keys = list(data_temp['key'].unique())

In [6]:
duration = list()

for key in keys:
    subtable = data_temp[data_temp['key'] == key]
    time_end = datetime.strptime((list(subtable['Время замера'])[-1]), '%Y-%m-%d %H:%M:%S')
    time_begin = datetime.strptime((list(subtable['Время замера'])[0]), '%Y-%m-%d %H:%M:%S')
    difference = time_end - time_begin
    duration.append(sum(int(i) * 60**index for index, i in enumerate(str(difference).split(":")[::-1])))

In [7]:
df['duration'] = duration

In [8]:
df['duration'].describe()

count     3216.000000
mean      1839.861629
std       1246.950298
min          0.000000
25%       1177.000000
50%       1581.000000
75%       2235.000000
max      23537.000000
Name: duration, dtype: float64

## Суммарное время нагрева электродами, то есть сумму значений по всем промежуткам между запусками нагрева электродов.

In [9]:
data_arc.head(10)

,key,Начало нагрева дугой,Конец нагрева дугой,Активная мощность,Реактивная мощность
0,1,2019-05-03 11:02:14,2019-05-03 11:06:02,0.976059,0.687084
1,1,2019-05-03 11:07:28,2019-05-03 11:10:33,0.805607,0.520285
2,1,2019-05-03 11:11:44,2019-05-03 11:14:36,0.744363,0.498805
3,1,2019-05-03 11:18:14,2019-05-03 11:24:19,1.659363,1.062669
4,1,2019-05-03 11:26:09,2019-05-03 11:28:37,0.692755,0.414397
5,2,2019-05-03 11:34:14,2019-05-03 11:36:31,0.438063,0.283043
6,2,2019-05-03 11:38:50,2019-05-03 11:44:28,1.296415,0.892914
7,2,2019-05-03 11:46:19,2019-05-03 11:48:25,0.490377,0.305281
8,2,2019-05-03 11:49:48,2019-05-03 11:53:18,0.827743,0.516874
9,3,2019-05-03 12:06:54,2019-05-03 12:11:34,1.062053,0.671494


In [10]:
data_arc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14876 entries, 0 to 14875
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   key                   14876 non-null  int64  
 1   Начало нагрева дугой  14876 non-null  object 
 2   Конец нагрева дугой   14876 non-null  object 
 3   Активная мощность     14876 non-null  float64
 4   Реактивная мощность   14876 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 581.2+ KB


In [11]:
data_arc['Начало нагрева дугой'] = pd.to_datetime(data_arc['Начало нагрева дугой'], format='%Y-%m-%d %H:%M:%S') 
data_arc['Конец нагрева дугой'] = pd.to_datetime(data_arc['Конец нагрева дугой'], format='%Y-%m-%d %H:%M:%S') 

In [12]:
data_arc['Дельта'] = data_arc['Конец нагрева дугой'] - data_arc['Начало нагрева дугой']

In [13]:
summa = list()

for key in keys:
    subtable = data_arc[data_arc['key'] == key]
    summ = subtable['Дельта'].sum()
    summa.append(sum(int(i) * 60**index for index, i in enumerate(str(summ).replace(' days 0', '').split(":")[::-1])))

In [14]:
df['summa'] = summa

In [15]:
df['summa'].describe()

count    3216.000000
mean      794.084888
std       332.935196
min         0.000000
25%       571.000000
50%       769.500000
75%       983.000000
max      4189.000000
Name: summa, dtype: float64

## Количество запусков нагрева электродами.

In [16]:
kolvo = list()

for key in keys:
    subtable = data_arc[data_arc['key'] == key]
    kolvo.append(len(subtable))

In [17]:
df['launch count'] = kolvo

In [18]:
df['launch count'].describe()

count    3216.000000
mean        4.625622
std         1.612283
min         0.000000
25%         4.000000
50%         4.000000
75%         6.000000
max        16.000000
Name: launch count, dtype: float64

## Среднее соотношение потребления активной и реактивной мощности.

***Потребление (Сумма) = Активная мощность + Реактивная мощность***

In [19]:
data_arc['Потребление'] = data_arc['Активная мощность'] + data_arc['Реактивная мощность']

In [20]:
data_arc['active ratio'] = data_arc['Активная мощность'] / data_arc['Потребление']
data_arc['reactive ratio'] = data_arc['Реактивная мощность'] / data_arc['Потребление']

In [21]:
mean_active_ratio = list()
mean_reactive_ratio = list()

for key in keys:
    subtable = data_arc[data_arc['key'] == key]
    mean_active_ratio.append(subtable['active ratio'].mean())
    mean_reactive_ratio.append(subtable['reactive ratio'].mean())

In [22]:
df['mean active ratio'] = mean_active_ratio
df['mean reactive ratio'] = mean_reactive_ratio

In [23]:
print('mean active ratio')
display(df['mean active ratio'].describe())
print('\nmean reactive ratio')
display(df['mean reactive ratio'].describe())

mean active ratio


count    3214.000000
mean        0.573450
std         0.026255
min         0.397099
25%         0.563033
50%         0.577379
75%         0.589201
max         0.638808
Name: mean active ratio, dtype: float64


mean reactive ratio


count    3214.000000
mean        0.426550
std         0.026255
min         0.361192
25%         0.410799
50%         0.422621
75%         0.436967
max         0.602901
Name: mean reactive ratio, dtype: float64

## Итого и выводы

Все полученные результаты отражены в столбцах таблицы df.

Выше отдельно по каждому столбцу была применена команда **describe()** для определения таких статистических показателей, как средние, минимальные и максимальные значения, медиану и величины 25%- и 75%-квартилей. Для медианы используем отдельную команду median().

Ниже представлена итоговая таблица df, describe() для нее и median()...

In [25]:
display(df.head())

,duration,summa,launch count,mean active ratio,mean reactive ratio
0,861,1098,5,0.605710,0.394290
1,1305,811,4,0.607888,0.392112
2,1300,655,5,0.612851,0.387149
3,388,741,4,0.609497,0.390503
4,762,869,4,0.596545,0.403455


In [26]:
display(df.describe())

,duration,summa,launch count,mean active ratio,mean reactive ratio
count,3216.000000,3216.000000,3216.000000,3214.000000,3214.000000
mean,1839.861629,794.084888,4.625622,0.573450,0.426550
std,1246.950298,332.935196,1.612283,0.026255,0.026255
min,0.000000,0.000000,0.000000,0.397099,0.361192
25%,1177.000000,571.000000,4.000000,0.563033,0.410799
50%,1581.000000,769.500000,4.000000,0.577379,0.422621
75%,2235.000000,983.000000,6.000000,0.589201,0.436967
max,23537.000000,4189.000000,16.000000,0.638808,0.602901


In [28]:
display(df.median())

duration               1581.000000
summa                   769.500000
launch count              4.000000
mean active ratio         0.577379
mean reactive ratio       0.422621
dtype: float64